In [1]:
%reload_ext autoreload
%autoreload 2

# NumPy
import numpy as np

# torch
import torch
from torch.optim import lr_scheduler
from torch.utils.data import DataLoader, TensorDataset, random_split

# Misc.
from tqdm.auto import tqdm

from src import models
from src.utils import seed_everything
from src.utils.pde_utils import make_mesh

# user-defined libs.
from src.utils.plotting import Artist

# Reproducibility
seed_everything()
# Set Device(CPU / GPU)
torch.set_float32_matmul_precision("high")
torch.set_default_dtype(torch.float32)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"torch is using {torch.cuda.get_device_name(device)}")
# Metadata
log_dir = "logs"
task = "burgers"
artist = Artist()

torch is using NVIDIA RTX A4000


In [2]:
# Load whole data
data = np.load("data/Burgers_spectral_1024_512.npz")
xs = torch.Tensor(data["xs"])
ts = torch.Tensor(data["ts"])
ys = torch.Tensor(data["ys"])
coefficient = data["coefficient"]
print("Whole Dataset")
print("xs", xs.shape, type(xs))
print("ts", ts.shape, type(ts))
print("ys", ys.shape, type(ys))

Whole Dataset
xs torch.Size([1024]) <class 'torch.Tensor'>
ts torch.Size([512]) <class 'torch.Tensor'>
ys torch.Size([1000, 1024, 512]) <class 'torch.Tensor'>


In [3]:
# Create Train / Test Dataset
Nx, Nt = len(xs), len(ts)
num_x = 64
num_step = 32

u_data = ys[:, :: len(xs) // num_x, :num_step]
y_data = torch.tile(xs[:: len(xs) // num_x], (len(ys), 1)).unsqueeze(-1)
s_data = ys[:, :: len(xs) // num_x, :]

dataset = TensorDataset(u_data, y_data, s_data)
train_dataset, test_dataset = random_split(
    dataset, lengths=[0.8, 0.2], generator=torch.Generator().manual_seed(41)
)

print("u:", u_data.shape)
print("y:", y_data.shape)
print("s:", s_data.shape)
print("train data:", len(train_dataset), "test data:", len(test_dataset))

u: torch.Size([1000, 64, 32])
y: torch.Size([1000, 64, 1])
s: torch.Size([1000, 64, 512])
train data: 800 test data: 200


## Fourier Neural Operator 

In [4]:
# Set train parameter
lr = 1e-3
batch_size = 20
epochs = 500
log_interval = 10
model_name = "FNO1D"
model_log = f"{log_dir}/{model_name}/{task}"

model = models.FNO1D(
    num_step=num_step,
    n_dimension=1,
    modes=12,
    width=32,
    num_blocks=3,
    hidden_dim=128,
    activation="relu",
).to(device)


criterion = torch.nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=lr)
# scheduler = lr_scheduler.StepLR(optimizer, step_size=200, gamma=0.5)
scheduler = lr_scheduler.ReduceLROnPlateau(
    optimizer,
    mode="min",
    factor=0.5,
    patience=5,
    threshold=1e-4,
)

train_loader = DataLoader(train_dataset, batch_size, shuffle=True, num_workers=6)
test_loader = DataLoader(test_dataset, batch_size, shuffle=False)


pbar = tqdm(range(epochs), desc="Training")
step = 0
for e in pbar:
    # Train model
    model.train()
    for batch_idx, (u, y, s) in enumerate(train_loader):
        optimizer.zero_grad()
        u = u.to(device)
        y = y.to(device)
        s = s.to(device)

        preds = model(u, y, len(ts))
        loss = criterion(preds.view(-1, 1), s.view(-1, 1))
        loss.backward()
        optimizer.step()

        # Log ouptut at each interval
        if (step + 1) % log_interval == 0:
            pbar.set_description(
                "Train Epoch(step: {:05d}): {:4d} [{:06d}/{:06d} ({:3.0f}%)] Train Loss: {:.6f}".format(
                    step + 1,
                    e,
                    batch_idx * len(u),
                    len(train_loader.dataset),
                    100.0 * batch_idx / len(train_loader),
                    loss.item(),
                )
            )
            with torch.no_grad():
                plot_u = ys[-1, :, :num_step].to(device)
                plot_y = xs.unsqueeze(1).to(device)
                plot_s = ys[-1]
                artist.plot_pde(
                    ts,
                    xs,
                    plot_s,
                    model(plot_u, plot_y, len(ts)).view_as(plot_s).detach().cpu(),
                    f"{model_name} / Training Step: {step+1:04d}, Loss:{loss.item():.3f}",
                )
                artist.save_img(f"step_{step+1:05d}", f"{model_log}/imgs")
        step += 1

    # # Test model
    # test_loss = 0
    # model.eval()
    # with torch.no_grad():
    #     for batch_idx, (u, y, s) in enumerate(test_loader):
    #         u = u.to(device)
    #         y = y.to(device)
    #         s = s.to(device)

    #         preds = model(u, y, len(ts))
    #         loss = criterion(preds.view(-1, 1), s.view(-1, 1)) * len(preds)
    #         test_loss += loss.item()
    #     # pbar.set_description("Test  Epoch: {:4d} Test Loss: {:.6f}".format(e, test_loss / len(test_loader.dataset)))
    scheduler.step(loss)
artist.save_gif_from_files(f"{model_name}-{task}", model_log)

Training:   0%|          | 0/500 [00:00<?, ?it/s]

: 